# Solución para la predicción de la diabetes

## Modelo de refuerzo

### Opciones y paquetes

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config IPCompleter.greedy=True  # Automcompletado rápido
%matplotlib inline
# Asegurarse de que matplotlib use el backend inline para mostrar gráficos en Jupyter
# Hace que los gráficos generados con matplotlib se muestren directamente dentro del notebook, justo debajo de la celda que los genera.



### Importación de datos

In [ ]:
train_data = pd.read_csv("Teorico/1. Decision_Tree/data/processed/clean_train.csv")
test_data = pd.read_csv("Teorico/1. Decision_Tree/data/processed/clean_test.csv")

train_data.head()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2.0,84.0,0.0,0.0,0.0,0.3,21.0,0
1,9.0,112.0,82.0,0.0,28.2,1.3,50.0,1
2,1.0,139.0,46.0,83.0,28.7,0.7,22.0,0
3,0.0,161.0,50.0,0.0,21.9,0.3,65.0,0
4,6.0,134.0,80.0,370.0,46.2,0.2,46.0,1


#### Separar predictoras y target

#### Separar train y test

Importamos el df del proyecto de decision tree, ya separados en train y test

In [3]:
X_train = train_data.drop(["Outcome"], axis = 1)
y_train = train_data["Outcome"]
X_test = test_data.drop(["Outcome"], axis = 1)
y_test = test_data["Outcome"]

In [1]:
import xgboost
print(xgboost.__version__)  # Recomendado: >= 1.3

3.0.0


In [6]:
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# ◯ Paso 1: Crear diccionario para guardar el historial de métricas
evals_result = {}

# ◯ Paso 2: Instanciar el clasificador XGBoost
        # ✓ use_label_encoder=False: Evita advertencias de deprecación
        # ✓ eval_metric='error': Usamos tasa de error como métrica donde 'error' = 1 - accuracy
        # ✓ random_state=42: Para reproducibilidad

xgb = XGBClassifier(use_label_encoder=False, eval_metric='error',  random_state=42)

# ◯ Paso 3: Entrenar el modelo usando 'error' como métrica (tasa de error)
xgb.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],     # ✓ Evaluamos sobre conjunto de validación
    early_stopping_rounds=5,         # ✓ Detiene si no mejora después de 5 iteraciones
    verbose=True ,                  # ✓ Muestra información de progreso durante el entrenamiento
    evals_result=evals_result       # ✓ Guardar el historial de métricas en evals_result
)

# ◯ Paso 4: Obtener el historial de métricas desde el modelo
evals_result = xgb.evals_result()

# ◯ Paso 5: Convertir el error en accuracy
errors = evals_result['validation_0']['error']
accuracies = [1 - e for e in errors]  # ✓ Accuracy = 1 - error

# ◯ Paso 6: Graficar evolución del Accuracy por iteración
plt.figure(figsize=(8, 4))
plt.plot(accuracies, marker='o')
plt.title('Accuracy por iteración (conjunto de validación)')
plt.xlabel('Iteración')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()


TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

In [ ]:
from xgboost import XGBClassifier

# Instanciar el clasificador y configurar los parámetros
xgb = XGBClassifier( eval_metric='auc', early_stopping_rounds=5)

# Entrenar
xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)])


[0]	validation_0-auc:0.78815
[1]	validation_0-auc:0.80257
[2]	validation_0-auc:0.80349
[3]	validation_0-auc:0.79816
[4]	validation_0-auc:0.78522
[5]	validation_0-auc:0.78641
[6]	validation_0-auc:0.78760


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators = 800, max_depth=5, learning_rate = 0.001, random_state = 42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.001, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=800,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7792207792207793

Este algoritmo de refuerzo es mejor que el árbol de decisión optimizado.

In [ ]:
from pickle import dump

dump(model, open("models/boosting_classifier_nestimators-20_learnrate-0.001_42.sav", "wb"))

In [ ]:
import xgboost
print(xgboost.__version__)


3.0.0
